# Some GUI code

In [169]:
from tkinter.filedialog  import askdirectory, askopenfilename, asksaveasfilename
from os import getcwd

root = tkinter.Tk()
root.withdraw()

def ask_open_directory(initialdir=None, title='Select a folder'):
    return askdirectory(parent=root, 
                        initialdir=(initialdir or getcwd()), 
                        title=title)

def ask_open_file (initialdir=None, title='Select a file', filetypes=None):
    return askopenfilename(parent=root, 
                           initialdir=getcwd(), 
                           title=title, 
                           filetypes=filetypes)

def ask_save_file(initialdir=None, title='Save as', confirmoverwrite=True, defaultextension='.txt'):
    return asksaveasfilename(initialdir=None, 
                             title=title,
                             confirmoverwrite=confirmoverwrite, 
                             defaultextension=defaultextension)

def pretty_df(df):
    from IPython.display import HTML
    display(HTML(df.to_html()))
    return

# Some file system code

In [158]:
from os import walk
from os.path import join, splitext, abspath
import pandas as pd
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', 200)

def get_ext(fn):
    ext = splitext(fn)[1]
    return ext.lower().strip('.') if format else None

def is_raster(fn, test=False):
    supported_formats = ["tif"]
    return get_ext(fn) in supported_formats

def is_vector(fn, test=False):
    supported_formats = ["shp"]
    return get_ext(fn) in supported_formats

def get_datatype(fn):
    x = is_raster(fn)
    if x:
        return x
    x = is_vector(fn)
    if x:
        return x
    return None

def get_files_framed(path):
    data = [(filename, join(abspath(root), filename)) for root, dirs, files in os.walk(path) for filename in files]
    df = pd.DataFrame(data, columns=['filename', 'fullpath'])
    return df

def get_geodata_framed(df):
    df['datatype'] = df['filename'].apply(get_datatype)
    return geo_df

def list_rasters(df):
    dfr = df.copy()
    dfr['raster'] = dfr.filename.apply(lambda x: is_raster(x))
    dfr = dfr[dfr.raster==True].reset_index(drop=True)
    dfr.index += 1
    return dfr

def list_vectors(df):
    supported_formats = ["tif"]
    dfv = df.copy()
    dfv['vector'] = dfv.filename.apply(lambda x: is_vector(x))
    dfv = dfv[dfv.vector==True].reset_index(drop=True)
    dfv.index += 1
    return dfv

def list_geodata(path, rasters=False, vectors=False):
    files_df = get_files_framed(path)
    d = {}
    if rasters:
        d['rasters'] = list_rasters(files_df)
    if rasters:
        d['vectors'] = list_vectors(files_df)
    return d


# List spatial data in a folder

In [188]:
data_home = ask_directory(title='Select a data workspace')
print(f"\ndata_home = {data_home}")
if data_home:
    gd = list_geodata(data_home, rasters=True, vectors=True)
    pretty_df(gd['vectors'])
    pretty_df(gd['rasters'])


data_home = C:/Users/byed/spatial/spatial_data/grid-garage-sample-data-master


,filename,fullpath,vector
1,HydroLine_All_Watercourses.shp,C:\Users\byed\spatial\spatial_data\grid-garage-sample-data-master\StudyArea_Armidale\HydroLine_All_Watercourses.shp,True
2,mcass_studyareatemplate30m_sml.shp,C:\Users\byed\spatial\spatial_data\grid-garage-sample-data-master\StudyArea_Armidale\mcass_studyareatemplate30m_sml.shp,True
3,Place_Names.shp,C:\Users\byed\spatial\spatial_data\grid-garage-sample-data-master\StudyArea_Armidale\Place_Names.shp,True
4,Statewide_RiverConditionIndex.shp,C:\Users\byed\spatial\spatial_data\grid-garage-sample-data-master\StudyArea_Armidale\Statewide_RiverConditionIndex.shp,True
5,VegBorderRiversGwydir_Comp09_VIS3801.shp,C:\Users\byed\spatial\spatial_data\grid-garage-sample-data-master\StudyArea_Armidale\VegBorderRiversGwydir_Comp09_VIS3801.shp,True


,filename,fullpath,raster
1,demfill25m.tif,C:\Users\byed\spatial\spatial_data\grid-garage-sample-data-master\StudyArea_Armidale\demfill25m.tif,True
2,MCASS_StudyAreaTemplate30m_sml.tif,C:\Users\byed\spatial\spatial_data\grid-garage-sample-data-master\StudyArea_Armidale\MCASS_StudyAreaTemplate30m_sml.tif,True
3,netvegkc100m_sml.tif,C:\Users\byed\spatial\spatial_data\grid-garage-sample-data-master\StudyArea_Armidale\netvegkc100m_sml.tif,True
4,rain_ann1km.tif,C:\Users\byed\spatial\spatial_data\grid-garage-sample-data-master\StudyArea_Armidale\rain_ann1km.tif,True
5,sheep_dse1km.tif,C:\Users\byed\spatial\spatial_data\grid-garage-sample-data-master\StudyArea_Armidale\sheep_dse1km.tif,True
6,SPOT5_WoodyExtent_Armidale_sml.TIF,C:\Users\byed\spatial\spatial_data\grid-garage-sample-data-master\StudyArea_Armidale\SPOT5_WoodyExtent_Armidale_sml.TIF,True


## Save vectors to a csv table

In [173]:
f = ask_save_file(defaultextension='.csv')
if f:
    gd['vectors'].to_csv(f)
    print(f)

## Save rasters to a csv table

In [175]:
f = ask_save_file(defaultextension='.csv')
if f:
    gd['rasters'].to_csv(f)
    print(f)

C:/Users/byed/spatial/spatial_data/dev/ras.csv


# Vectors to Rasters

## Select a template raster

In [176]:
template = ask_file(title='Please select a raster to use as a project template', filetypes=[('GeoTIFF', '*.tif')])
print(template)

C:/Users/byed/spatial/spatial_data/grid-garage-sample-data-master/StudyArea_Armidale/MCASS_StudyAreaTemplate30m_sml.tif


## Select an output folder

In [180]:
out_folder = ask_open_directory(title='Select a folder for raster outputs')
print(out_folder)

C:/Users/byed/spatial/spatial_data/dev


## Map input and output filenames

In [201]:
v = gd['vectors'].copy()
v.drop('vector', axis=1, inplace=True)
v.rename(columns={'fullpath': 'in_vector'}, inplace=True)
v['out_raster'] = v['filename'].apply(lambda x: join(out_folder, f'{x}.tif'))
pretty_df(v)

,filename,in_vector,out_raster
1,HydroLine_All_Watercourses.shp,C:\Users\byed\spatial\spatial_data\grid-garage-sample-data-master\StudyArea_Armidale\HydroLine_All_Watercourses.shp,C:/Users/byed/spatial/spatial_data/dev\HydroLine_All_Watercourses.shp.tif
2,mcass_studyareatemplate30m_sml.shp,C:\Users\byed\spatial\spatial_data\grid-garage-sample-data-master\StudyArea_Armidale\mcass_studyareatemplate30m_sml.shp,C:/Users/byed/spatial/spatial_data/dev\mcass_studyareatemplate30m_sml.shp.tif
3,Place_Names.shp,C:\Users\byed\spatial\spatial_data\grid-garage-sample-data-master\StudyArea_Armidale\Place_Names.shp,C:/Users/byed/spatial/spatial_data/dev\Place_Names.shp.tif
4,Statewide_RiverConditionIndex.shp,C:\Users\byed\spatial\spatial_data\grid-garage-sample-data-master\StudyArea_Armidale\Statewide_RiverConditionIndex.shp,C:/Users/byed/spatial/spatial_data/dev\Statewide_RiverConditionIndex.shp.tif
5,VegBorderRiversGwydir_Comp09_VIS3801.shp,C:\Users\byed\spatial\spatial_data\grid-garage-sample-data-master\StudyArea_Armidale\VegBorderRiversGwydir_Comp09_VIS3801.shp,C:/Users/byed/spatial/spatial_data/dev\VegBorderRiversGwydir_Comp09_VIS3801.shp.tif


## Rasterise the vector shapes

In [178]:
import sys
!conda install --yes --prefix {sys.prefix} geopandas


CondaHTTPError: HTTP 000 CONNECTION FAILED for url <https://repo.anaconda.com/pkgs/msys2/noarch/repodata.json.bz2>
Elapsed: -

An HTTP error occurred when trying to retrieve this URL.
HTTP errors are often intermittent, and a simple retry will get you on your way.

If your current network has https://www.anaconda.com blocked, please file
a support request with your network engineering team.

ConnectionError(MaxRetryError("HTTPSConnectionPool(host='repo.anaconda.com', port=443): Max retries exceeded with url: /pkgs/msys2/noarch/repodata.json.bz2 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000285DC4C2B00>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))",),)




In [ ]:
import geopandas as gpd
import rasterio
from rasterio import features

